In [ ]:
# ============================================================================
# HYBRID FORM FILLER: LANGCHAIN FOR FIELD IDENTIFICATION + INSTRUCTOR FOR I/O
# ============================================================================

import asyncio
from typing import Dict, List, Optional, Tuple, Any, Union, Literal
from dataclasses import dataclass, field
from enum import Enum
import json
import os
from datetime import datetime
import logging
import instructor
from pydantic import BaseModel, Field
import anthropic

# LangChain imports
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain.tools import Tool, StructuredTool
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.base import BaseCallbackHandler

# Playwright
from playwright.async_api import Page, Browser, ElementHandle, Locator, async_playwright

# Standard library
from dotenv import load_dotenv
load_dotenv()

# ============================================================================
# INSTRUCTOR MODELS FOR STRUCTURED DATA
# ============================================================================

class ElementType(str, Enum):
    """Types of form elements"""
    TEXT_INPUT = "text_input"
    EMAIL_INPUT = "email"
    PASSWORD_INPUT = "password"
    TEXTAREA = "textarea"
    SELECT = "select"
    RADIO = "radio"
    CHECKBOX = "checkbox"
    NUMBER_INPUT = "number"
    DATE_INPUT = "date"
    CUSTOM_DROPDOWN = "custom_dropdown"
    UNKNOWN = "unknown"

class FormElement(BaseModel):
    """Structured representation of a form element"""
    selector: str = Field(description="CSS selector for the element")
    element_type: ElementType = Field(description="Type of form element")
    name: Optional[str] = Field(description="Name attribute")
    label: Optional[str] = Field(description="Associated label text")
    placeholder: Optional[str] = Field(description="Placeholder text")
    current_value: Optional[str] = Field(description="Current value if any")
    is_required: bool = Field(default=False, description="Whether field is required")
    options: Optional[List[str]] = Field(default=None, description="Options for select/radio")

class FieldMapping(BaseModel):
    """Mapping between data field and form element"""
    data_field: str = Field(description="Field name from input data")
    element_selector: str = Field(description="CSS selector for form element")
    element_type: ElementType = Field(description="Type of element")
    confidence: float = Field(ge=0.0, le=1.0, description="Confidence in mapping")
    transformation: Optional[str] = Field(default=None, description="Any transformation needed")
    reasoning: str = Field(description="Why this mapping makes sense")

class FormAction(BaseModel):
    """A single action to perform on the form"""
    action_type: Literal["fill", "select", "click", "check", "uncheck"] = Field(description="Type of action")
    selector: str = Field(description="Element selector")
    value: str = Field(description="Value to use")
    element_type: ElementType = Field(description="Type of element")
    order: int = Field(description="Order of execution (lower first)")
    pre_action: Optional[str] = Field(default=None, description="Action to perform before main action")
    post_action: Optional[str] = Field(default=None, description="Action to perform after main action")

class FormFillingPlan(BaseModel):
    """Complete plan for filling a form"""
    form_purpose: str = Field(description="What this form is for")
    actions: List[FormAction] = Field(description="Ordered list of actions to perform")
    validation_steps: List[str] = Field(description="Steps to validate after filling")
    warnings: List[str] = Field(default_factory=list, description="Potential issues to watch for")

class ExecutionResult(BaseModel):
    """Result of executing a form action"""
    action: FormAction
    success: bool
    result_message: str
    actual_value: Optional[str] = None
    error: Optional[str] = None
    timestamp: datetime = Field(default_factory=datetime.now)

class FormAnalysisResult(BaseModel):
    """Result of analyzing a form"""
    form_elements: List[FormElement]
    form_purpose: str
    recommended_mappings: Dict[str, str]
    element_count: int
    has_required_fields: bool

# ============================================================================
# PLAYWRIGHT AUTOMATION (Direct Execution)
# ============================================================================

class PlaywrightExecutor:
    """Handles direct Playwright execution - no AI decisions here"""
    
    def __init__(self):
        self.playwright = None
        self.browser = None
        self.page = None
        self.context = None
        
    async def initialize(self):
        """Initialize Playwright browser"""
        self.playwright = await async_playwright().start()
        self.browser = await self.playwright.chromium.launch(
            headless=False,
            args=['--no-sandbox', '--disable-blink-features=AutomationControlled']
        )
        self.context = await self.browser.new_context(
            viewport={'width': 1280, 'height': 720},
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        )
        self.page = await self.context.new_page()
        print("✅ Browser initialized")
        return self.page
        
    async def close(self):
        """Close browser and cleanup"""
        if self.browser:
            await self.browser.close()
        if self.playwright:
            await self.playwright.stop()
    
    async def navigate(self, url: str) -> bool:
        """Navigate to URL"""
        try:
            response = await self.page.goto(url, wait_until="domcontentloaded")
            await asyncio.sleep(2)
            return response.status == 200
        except Exception as e:
            print(f"Navigation error: {e}")
            return False
    
    async def extract_form_elements(self) -> List[Dict[str, Any]]:
        """Extract all form elements as raw data"""
        elements = []
        
        # Comprehensive selectors
        selectors = [
            "input:not([type='hidden']):not([type='submit']):not([type='button'])",
            "textarea",
            "select",
            "[role='combobox']",
            "[role='textbox']",
            "[contenteditable='true']"
        ]
        
        for selector in selectors:
            try:
                located_elements = await self.page.locator(selector).all()
                
                for element in located_elements:
                    if not await element.is_visible():
                        continue
                        
                    # Extract element data
                    elem_data = {
                        "selector": "",
                        "tag": await element.evaluate("el => el.tagName.toLowerCase()"),
                        "type": await element.get_attribute("type") or "text",
                        "id": await element.get_attribute("id"),
                        "name": await element.get_attribute("name"),
                        "class": await element.get_attribute("class"),
                        "placeholder": await element.get_attribute("placeholder"),
                        "aria-label": await element.get_attribute("aria-label"),
                        "required": await element.get_attribute("required") is not None,
                        "value": None,
                        "options": []
                    }
                    
                    # Build selector
                    if elem_data["id"]:
                        elem_data["selector"] = f"#{elem_data['id']}"
                    elif elem_data["name"]:
                        elem_data["selector"] = f"{elem_data['tag']}[name='{elem_data['name']}']"
                    else:
                        continue
                    
                    # Get current value
                    try:
                        elem_data["value"] = await element.input_value()
                    except:
                        elem_data["value"] = await element.text_content()
                    
                    # Get options for select
                    if elem_data["tag"] == "select":
                        options = await element.locator("option").all()
                        elem_data["options"] = [await opt.text_content() for opt in options]
                    
                    # Get label
                    if elem_data["id"]:
                        label = self.page.locator(f"label[for='{elem_data['id']}']")
                        if await label.count() > 0:
                            elem_data["label"] = await label.text_content()
                    
                    elements.append(elem_data)
                    
            except Exception as e:
                print(f"Error extracting {selector}: {e}")
                continue
                
        return elements
    
    async def execute_action(self, action: FormAction) -> ExecutionResult:
        """Execute a single form action"""
        try:
            element = self.page.locator(action.selector).first
            
            # Pre-action if needed
            if action.pre_action:
                if action.pre_action == "click_to_enable":
                    await element.click()
                    await asyncio.sleep(0.5)
                elif action.pre_action == "clear":
                    await element.clear()
            
            # Main action execution
            if action.action_type == "fill":
                await element.wait_for(state="visible", timeout=5000)
                await element.click()
                await element.fill(action.value)
                
                # Verify
                actual = await element.input_value()
                success = action.value in actual
                
            elif action.action_type == "select":
                if action.element_type == ElementType.SELECT:
                    # Standard select
                    await element.select_option(label=action.value)
                    success = True
                elif action.element_type == ElementType.CUSTOM_DROPDOWN:
                    # Custom dropdown
                    await element.click()
                    await asyncio.sleep(0.5)
                    option = self.page.locator(f"text='{action.value}'").first
                    await option.click()
                    success = True
                else:
                    success = False
                    
            elif action.action_type == "click":
                await element.click()
                success = True
                
            elif action.action_type == "check":
                is_checked = await element.is_checked()
                if not is_checked:
                    await element.click()
                success = True
                
            elif action.action_type == "uncheck":
                is_checked = await element.is_checked()
                if is_checked:
                    await element.click()
                success = True
            else:
                success = False
            
            # Post-action if needed
            if action.post_action:
                if action.post_action == "tab":
                    await self.page.keyboard.press("Tab")
                elif action.post_action == "enter":
                    await self.page.keyboard.press("Enter")
            
            return ExecutionResult(
                action=action,
                success=success,
                result_message=f"Successfully executed {action.action_type} on {action.selector}" if success else f"Failed to execute {action.action_type}",
                actual_value=await element.input_value() if action.action_type == "fill" else None
            )
            
        except Exception as e:
            return ExecutionResult(
                action=action,
                success=False,
                result_message=f"Error executing action",
                error=str(e)
            )

# ============================================================================
# LANGCHAIN FIELD IDENTIFIER
# ============================================================================

class LangChainFieldIdentifier:
    """Uses LangChain to identify which fields map to which data"""
    
    def __init__(self, llm: ChatAnthropic):
        self.llm = llm
        self.tools = self._create_tools()
        self.agent = None
        self.current_form_elements = []
        self.mappings = []
        
    def _create_tools(self):
        """Create tools for field identification"""
        
        def analyze_field_purpose(field_description: str) -> str:
            """Analyze what a form field is for based on its attributes"""
            # Find matching element
            for elem in self.current_form_elements:
                if field_description in str(elem):
                    return f"Field appears to be for: {elem.get('name', 'unknown')} with type {elem.get('type', 'unknown')}"
            return "Field not found in current form"
        
        def match_data_to_field(data_field: str, field_selector: str, confidence: str, reasoning: str) -> str:
            """Record a mapping between data field and form element"""
            try:
                conf = float(confidence)
                # Find element details
                element = None
                for elem in self.current_form_elements:
                    if elem.get('selector') == field_selector:
                        element = elem
                        break
                
                if element:
                    mapping = FieldMapping(
                        data_field=data_field,
                        element_selector=field_selector,
                        element_type=self._determine_element_type(element),
                        confidence=conf,
                        reasoning=reasoning
                    )
                    self.mappings.append(mapping)
                    return f"Mapped {data_field} to {field_selector} with confidence {conf}"
                else:
                    return f"Could not find element with selector {field_selector}"
            except Exception as e:
                return f"Error creating mapping: {str(e)}"
        
        def list_available_fields() -> str:
            """List all available form fields"""
            if not self.current_form_elements:
                return "No form elements loaded"
            
            fields = []
            for elem in self.current_form_elements:
                field_desc = f"- {elem['selector']}: {elem['type']}"
                if elem.get('label'):
                    field_desc += f" (label: {elem['label']})"
                if elem.get('placeholder'):
                    field_desc += f" (placeholder: {elem['placeholder']})"
                fields.append(field_desc)
            
            return "Available form fields:\n" + "\n".join(fields)
        
        return [
            Tool(
                name="analyze_field_purpose",
                func=analyze_field_purpose,
                description="Analyze what a form field is for based on its description"
            ),
            Tool(
                name="match_data_to_field",
                func=match_data_to_field,
                description="Map a data field to a form element. Args: data_field, field_selector, confidence (0-1), reasoning"
            ),
            Tool(
                name="list_available_fields",
                func=list_available_fields,
                description="List all available form fields with their attributes"
            )
        ]
    
    def _determine_element_type(self, element: Dict) -> ElementType:
        """Determine element type from raw data"""
        tag = element.get('tag', '')
        input_type = element.get('type', '')
        
        if tag == 'textarea':
            return ElementType.TEXTAREA
        elif tag == 'select':
            return ElementType.SELECT
        elif tag == 'input':
            if input_type == 'email':
                return ElementType.EMAIL_INPUT
            elif input_type == 'password':
                return ElementType.PASSWORD_INPUT
            elif input_type == 'number':
                return ElementType.NUMBER_INPUT
            elif input_type == 'date':
                return ElementType.DATE_INPUT
            elif input_type == 'checkbox':
                return ElementType.CHECKBOX
            elif input_type == 'radio':
                return ElementType.RADIO
            else:
                return ElementType.TEXT_INPUT
        else:
            return ElementType.UNKNOWN
    
    def create_agent(self):
        """Create the LangChain agent for field identification"""
        prompt = PromptTemplate(
            input_variables=["input", "tools", "tool_names", "agent_scratchpad"],
            template="""You are an expert at identifying form fields and mapping data to them.

    Available tools: {tools}
    Tool names: {tool_names}

    You must use the following format:

    Thought: I need to analyze the form and create mappings
    Action: the action to take, must be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I have completed all mappings
    Final Answer: Mappings complete

    Begin! Remember to ALWAYS use the exact format above.

    Current task: {input}

    {agent_scratchpad}"""
        )
        
        self.agent = create_react_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=prompt
        )
        
        return AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
            max_iterations=15,
            handle_parsing_errors=True,  # Add this line
            error_message="I need to use the proper format:\nThought: <reasoning>\nAction: <tool_name>\nAction Input: <input>"
        )


class InstructorFieldIdentifier:
    """Uses Instructor for structured field identification"""
    
    def __init__(self, instructor_client):
        self.client = instructor_client
        
    async def identify_mappings(self, form_elements: List[Dict], data_to_fill: Dict[str, str]) -> List[FieldMapping]:
        """Use Instructor to identify field mappings"""
        
        # Format form elements for prompt
        elements_desc = []
        for elem in form_elements[:20]:  # Limit to prevent token overflow
            desc = f"- Selector: {elem['selector']}"
            if elem.get('label'):
                desc += f", Label: {elem['label']}"
            if elem.get('placeholder'):
                desc += f", Placeholder: {elem['placeholder']}"
            if elem.get('name'):
                desc += f", Name: {elem['name']}"
            desc += f", Type: {elem['type']}"
            elements_desc.append(desc)
        
        # Create structured mapping request
        class MappingResponse(BaseModel):
            mappings: List[FieldMapping] = Field(description="List of field mappings")
            unmapped_fields: List[str] = Field(description="Data fields that couldn't be mapped")
            
        prompt = f"""Analyze these form elements and create mappings for the provided data.

Form Elements:
{chr(10).join(elements_desc)}

Data to map:
{json.dumps(data_to_fill, indent=2)}

For each data field, find the most appropriate form element based on:
1. Field names/IDs (cardName → name)
2. Labels and placeholders
3. Element types (textarea for long text, select for choices)
4. Common patterns (mana/cost, power/toughness pairs)

Create mappings with confidence levels:
- 0.9-1.0: Perfect match (exact ID/name match)
- 0.7-0.9: Good match (semantic similarity)
- 0.5-0.7: Possible match (type appropriate)
- Below 0.5: Skip the mapping
"""
        
        response = await asyncio.to_thread(
            self.client.chat.completions.create,
            model="claude-3-5-sonnet-20241022",
            response_model=MappingResponse,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1500
        )
        
        return response.mappings
    
# ============================================================================
# INSTRUCTOR-BASED ORCHESTRATOR
# ============================================================================

class FormFillingOrchestrator:
    """Orchestrates the form filling process using Instructor for structured I/O"""
    
    def __init__(self, anthropic_api_key: str):
        self.anthropic_client = anthropic.Anthropic(api_key=anthropic_api_key)
        self.instructor_client = instructor.from_anthropic(self.anthropic_client)
        self.llm = ChatAnthropic(
            model="claude-3-5-sonnet-20241022",
            api_key=anthropic_api_key,
            temperature=0
        )
        self.executor = PlaywrightExecutor()
        self.field_identifier = self.field_identifier = InstructorFieldIdentifier(self.instructor_client)
        
    async def initialize(self):
        """Initialize components"""
        await self.executor.initialize()
        
    async def analyze_form(self, form_elements: List[Dict]) -> FormAnalysisResult:
        """Use Instructor to analyze form structure"""
        
        # Convert raw elements to structured format
        structured_elements = []
        for elem in form_elements:
            structured_elements.append(FormElement(
                selector=elem['selector'],
                element_type=self._determine_element_type(elem),
                name=elem.get('name'),
                label=elem.get('label'),
                placeholder=elem.get('placeholder'),
                current_value=elem.get('value'),
                is_required=elem.get('required', False),
                options=elem.get('options', [])
            ))
        
        # Create analysis prompt
        prompt = f"""Analyze this form structure:

Elements found: {len(structured_elements)}
Types: {[e.element_type.value for e in structured_elements]}

Determine:
1. What is this form for?
2. Which fields are most important?
3. Are there any required fields?

Elements:
{json.dumps([e.model_dump() for e in structured_elements[:10]], indent=2)}
"""
        
        # Get structured analysis
        response = await asyncio.to_thread(
            self.instructor_client.chat.completions.create,
            model="claude-3-5-sonnet-20241022",
            response_model=FormAnalysisResult,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000
        )
        
        response.form_elements = structured_elements
        return response
    
    def _determine_element_type(self, element: Dict) -> ElementType:
        """Determine element type from raw data"""
        tag = element.get('tag', '')
        input_type = element.get('type', '')
        
        if tag == 'textarea':
            return ElementType.TEXTAREA
        elif tag == 'select':
            return ElementType.SELECT
        elif tag == 'input':
            if input_type == 'email':
                return ElementType.EMAIL_INPUT
            elif input_type == 'password':
                return ElementType.PASSWORD_INPUT
            elif input_type == 'number':
                return ElementType.NUMBER_INPUT
            elif input_type == 'checkbox':
                return ElementType.CHECKBOX
            elif input_type == 'radio':
                return ElementType.RADIO
            else:
                return ElementType.TEXT_INPUT
        else:
            return ElementType.UNKNOWN
    
    async def create_filling_plan(self, 
                                 mappings: List[FieldMapping], 
                                 form_analysis: FormAnalysisResult,
                                 data_to_fill: Dict[str, str]) -> FormFillingPlan:
        """Use Instructor to create execution plan"""
        
        # Format mappings for prompt
        mapping_desc = []
        for m in mappings:
            mapping_desc.append(f"- {m.data_field} → {m.element_selector} ({m.element_type.value}, confidence: {m.confidence})")
        
        prompt = f"""Create a form filling plan based on these mappings:

Form Purpose: {form_analysis.form_purpose}

Mappings:
{chr(10).join(mapping_desc)}

Data to fill:
{json.dumps(data_to_fill, indent=2)}

Create a detailed plan with:
1. Ordered list of actions (fill fields in logical order)
2. Any special handling needed for specific fields
3. Validation steps to ensure success
"""
        
        response = await asyncio.to_thread(
            self.instructor_client.chat.completions.create,
            model="claude-3-5-sonnet-20241022",
            response_model=FormFillingPlan,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1500
        )
        
        # Convert mappings to actions
        actions = []
        for i, mapping in enumerate(mappings):
            if mapping.confidence > 0.3 and mapping.data_field in data_to_fill:
                action = FormAction(
                    action_type="fill" if mapping.element_type != ElementType.SELECT else "select",
                    selector=mapping.element_selector,
                    value=data_to_fill[mapping.data_field],
                    element_type=mapping.element_type,
                    order=i
                )
                actions.append(action)
        
        response.actions = sorted(actions, key=lambda x: x.order)
        return response
    
    async def execute_plan(self, plan: FormFillingPlan) -> List[ExecutionResult]:
        """Execute the filling plan"""
        results = []
        
        print(f"\n⚡ Executing {len(plan.actions)} actions...")
        
        for i, action in enumerate(plan.actions):
            print(f"\n   Action {i+1}/{len(plan.actions)}: {action.action_type} on {action.selector}")
            
            result = await self.executor.execute_action(action)
            results.append(result)
            
            if result.success:
                print(f"   ✅ {result.result_message}")
            else:
                print(f"   ❌ {result.result_message}")
                if result.error:
                    print(f"      Error: {result.error}")
            
            await asyncio.sleep(0.5)  # Small delay between actions
        
        return results
    
    async def fill_form(self, url: str, data: Dict[str, str]) -> Dict[str, Any]:
        """Main method to fill a form"""
        
        print("\n🚀 Starting form filling process...")
        
        # Step 1: Navigate
        print("\n📍 Navigating to form...")
        success = await self.executor.navigate(url)
        if not success:
            return {"success": False, "error": "Failed to navigate to URL"}
        
        # Step 2: Extract form elements
        print("\n🔍 Extracting form elements...")
        form_elements = await self.executor.extract_form_elements()
        print(f"   Found {len(form_elements)} form elements")
        
        # Step 3: Analyze form with Instructor
        print("\n📊 Analyzing form structure...")
        form_analysis = await self.analyze_form(form_elements)
        print(f"   Form purpose: {form_analysis.form_purpose}")
        
        # Step 4: Identify field mappings with LangChain
        print("\n🎯 Identifying field mappings...")
        mappings = await self.field_identifier.identify_mappings(form_elements, data)
        print(f"   Created {len(mappings)} mappings")
        
        # Step 5: Create filling plan with Instructor
        print("\n📋 Creating execution plan...")
        plan = await self.create_filling_plan(mappings, form_analysis, data)
        print(f"   Plan contains {len(plan.actions)} actions")
        
        # Step 6: Execute plan
        results = await self.execute_plan(plan)
        
        # Step 7: Summary
        successful = sum(1 for r in results if r.success)
        print(f"\n✅ Completed: {successful}/{len(results)} actions successful")
        
        return {
            "success": successful > 0,
            "total_actions": len(results),
            "successful_actions": successful,
            "results": [r.model_dump() for r in results],
            "form_analysis": form_analysis.model_dump(),
            "mappings": [m.model_dump() for m in mappings],
            "plan": plan.model_dump()
        }
    
    async def close(self):
        """Cleanup"""
        await self.executor.close()

# ============================================================================
# CARD GENERATOR (Instructor)
# ============================================================================

class MTGCardGenerator:
    """Generate Magic cards using Instructor"""
    
    def __init__(self, anthropic_client):
        self.client = instructor.from_anthropic(anthropic_client)
        
    async def generate_card(self, concept: str) -> Dict[str, str]:
        """Generate a Magic card from a concept"""
        
        class MTGCard(BaseModel):
            name: str = Field(description="Card name")
            cost: str = Field(description="Mana cost (e.g., 3RR)")
            type: str = Field(description="Card type line")
            subtype: Optional[str] = Field(description="Card subtype")
            rules: str = Field(description="Rules text and abilities")
            power: Optional[str] = Field(description="Power (for creatures)")
            toughness: Optional[str] = Field(description="Toughness (for creatures)")
            rarity: str = Field(description="Rarity")
            
        prompt = f"Create a Magic: The Gathering card based on this concept: {concept}"
        
        response = await asyncio.to_thread(
            self.client.chat.completions.create,
            model="claude-3-5-sonnet-20241022",
            response_model=MTGCard,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500
        )
        
        # Convert to dict, excluding None values
        card_dict = {}
        for field, value in response.model_dump().items():
            if value is not None:
                card_dict[field] = value
                
        return card_dict

# ============================================================================
# MAIN EXECUTION
# ============================================================================

async def main():
    """Example usage"""
    
    # Check for API key
    api_key = os.getenv("ANTHROPIC_API_KEY")
    if not api_key:
        print("❌ Please set ANTHROPIC_API_KEY environment variable")
        return
        
    print("🚀 Hybrid Form Filler: LangChain + Instructor")
    print("="*60)
    
    orchestrator = None
    try:
        # Initialize Anthropic client
        anthropic_client = anthropic.Anthropic(api_key=api_key)
        
        # Generate card data
        print("\n🎴 Generating Magic card data...")
        generator = MTGCardGenerator(anthropic_client)
        card_data = await generator.generate_card("A dragon that controls time")
        
        print("\n📝 Generated Card:")
        for field, value in card_data.items():
            print(f"  - {field}: {value}")
        
        # Initialize orchestrator
        orchestrator = FormFillingOrchestrator(api_key)
        await orchestrator.initialize()
        
        # Fill the form
        results = await orchestrator.fill_form(
            "https://mtgcardsmith.com/mtg-card-maker/edit",
            card_data
        )
        
        # Display results
        print("\n" + "="*60)
        print("FORM FILLING RESULTS")
        print("="*60)
        print(f"✅ Success: {'Yes' if results['success'] else 'No'}")
        print(f"📊 Actions: {results['successful_actions']}/{results['total_actions']} successful")
        
        # Show mappings
        print("\n🎯 Field Mappings:")
        for mapping in results['mappings']:
            print(f"  - {mapping['data_field']} → {mapping['element_selector']} (confidence: {mapping['confidence']:.2f})")
        
        # Keep browser open
        print("\n✋ Browser will remain open for 30 seconds...")
        await asyncio.sleep(30)
        
    except KeyboardInterrupt:
        print("\n⚠️  Interrupted by user")
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
    finally:
        if orchestrator:
            await orchestrator.close()
        print("\n👋 Form filling session ended")

if __name__ == "__main__":
    asyncio.run(main())

🚀 Hybrid Form Filler: LangChain + Instructor

🎴 Generating Magic card data...

📝 Generated Card:
  - name: Chronoscale Dragon
  - cost: 4URR
  - type: Legendary Creature
  - subtype: Elder Dragon
  - rules: Flying

At the beginning of your upkeep, put a time counter on Chronoscale Dragon.

{T}: Take an extra turn after this one. Activate only if Chronoscale Dragon has three or more time counters on it. Remove all time counters from Chronoscale Dragon.
  - power: 6
  - toughness: 6
  - rarity: Mythic Rare
✅ Browser initialized

🚀 Starting form filling process...

📍 Navigating to form...

🔍 Extracting form elements...
   Found 26 form elements

📊 Analyzing form structure...
   Form purpose: Trading Card Game Card Creator Form

🎯 Identifying field mappings...
   Created 5 mappings

📋 Creating execution plan...
   Plan contains 5 actions

⚡ Executing 5 actions...

   Action 1/5: fill on #cardName
   ✅ Successfully executed fill on #cardName

   Action 2/5: fill on #custom_mana
   ✅ Success